In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import trange
import random
import math
from scipy import interp
import statistics 
import os
import torch

from vibtcr.dataset import TCRDataset, hard_split_df, get_balanced_torch_loader, get_st_scaler
from vibtcr.mvib.mvib import MVIB
from vibtcr.mvib.mvib_trainer import TrainerMVIB

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, auc

from matplotlib import collections
from matplotlib import colors
from numpy.random import normal

In [2]:
metrics = [
    'AUROC',
    'Accuracy',
    'Recall',
    'Precision',
    'F1 score',
    'AUPR'
]

def pr_auc(y_true, y_prob):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    pr_auc = auc(recall, precision)
    return pr_auc

def get_scores(y_true, y_prob, y_pred):
    """
    Compute a df with all classification metrics and respective scores.
    """
    
    scores = [
        roc_auc_score(y_true, y_prob),
        accuracy_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        f1_score(y_true, y_pred),
        pr_auc(y_true, y_prob)
    ]
    
    df = pd.DataFrame(data={'score': scores, 'metrics': metrics})
    return df

In [3]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

In [4]:
login = os.getlogin( )
DATA_BASE = f"/home/{login}/Git/tc-hard/tc-hard-data/tc-hard/ds.hard-splits/pep+cdr3b/"
RESULTS_BASE = f"/home/{login}/Git/tc-hard/notebooks/notebooks.classification/results/"

In [5]:
device = torch.device('cuda:0')
cpu_device = torch.device("cpu")

batch_size = 4096
epochs = 500
lr = 1e-3

z_dim = 150
early_stopper_patience = 50
monitor = 'auROC'
lr_scheduler_param = 10
joint_posterior = "aoe"

beta = 1e-6

# AVIB - Hard split (Test: only randomized negatives) - Train: only randomized negatives

In [6]:
results_avib = []

for i in tqdm(range(5)):
    set_random_seed(i)

    df_train = pd.read_csv(DATA_BASE+"train/only-sampled-negs/"+f"train-{i}.csv")
    df_test = pd.read_csv(DATA_BASE+"test/only-sampled-negs/"+f"test-{i}.csv")
    
    scaler = get_st_scaler(df_train, gt='label', cdr3b_col='cdr3.beta', pep_col="antigen.epitope")
    inner_train_df, val_df, _ = hard_split_df(
        df_train, target_col="antigen.epitope", min_ratio=0.15, low=500, high=10000
    )
    
    # train
    train_loader = get_balanced_torch_loader(inner_train_df, batch_size, gt='label', pep_col='antigen.epitope', cdr3b_col='cdr3.beta', device=device)
    val_loader = get_balanced_torch_loader(val_df, batch_size, gt='label', pep_col='antigen.epitope', cdr3b_col='cdr3.beta', device=device)

    model = MVIB(z_dim=z_dim, device=device, joint_posterior=joint_posterior).to(device)

    trainer = TrainerMVIB(
        model,
        epochs=epochs,
        lr=lr,
        beta=beta,
        checkpoint_dir=".",
        mode="p+b",
        lr_scheduler_param=lr_scheduler_param
    )
    checkpoint = trainer.train(train_loader, val_loader, early_stopper_patience, monitor)    
    
    # test
    ds_test = TCRDataset(
        df_test,
        torch.device("cpu"),
        cdr3b_col='cdr3.beta',
        pep_col="antigen.epitope",
        gt_col="label",
        cdr3a_col=None,
        scaler=scaler
    )
    model = MVIB.from_checkpoint(checkpoint, cpu_device).to(cpu_device).eval()
    pred = model.classify(pep=ds_test.pep, cdr3b=ds_test.cdr3b, cdr3a=None)
    pred = pred.detach().numpy()
    gt = ds_test.gt.cpu().numpy()
    
    scores_df = get_scores(y_true=gt, y_prob=pred, y_pred=pred.round())
    scores_df['experiment'] = i
    results_avib.append(scores_df)
    df_test['prediction'] = pred
    df_test.to_csv(RESULTS_BASE+f"avib.pep+cdr3b.only-sampled-negs.hard-split.{i}.csv", index=False)

results_avib_df = pd.concat(results_avib)
results_avib_df.to_csv(RESULTS_BASE+"avib.pep+cdr3b.only-sampled-negs.hard-split.csv", index=False)

  0%|          | 0/5 [00:00<?, ?it/s]/home/nle-fgrazioli/anaconda3/envs/vibtcr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)

  0%|          | 0/500 [00:00<?, ?it/s]
[VAL] Best epoch 1 (-0.495411) | DKL 0.000180 | BCE 0.693565 | auROC 0.4954 | auPR 0.4978:   0%|          | 0/500 [00:08<?, ?it/s]
[VAL] Best epoch 1 (-0.495411) | DKL 0.000180 | BCE 0.693565 | auROC 0.4954 | auPR 0.4978:   0%|          | 0/500 [00:08<?, ?it/s]
[VAL] Best epoch 1 (-0.495411) | DKL 0.000180 | BCE 0.693565 | auROC 0.4954 | auPR 0.4978:   0%|          | 1/500 [00:08<1:13:48,  8.88s/it]
[VAL] Best epoch 1 (-0.495411) | DKL 0.000180 | BCE 0.693565 | auROC 0.4954 | auPR 0.4978:   0%|          | 2/500 [00:17<1:13:02,  8.80s/it]
[VAL] Best epoch 3 (-0.502880) | DKL 0.000254 | BCE 0.694368 | auROC 0.5029 | auPR 0.4891:   0%|   

[VAL] Best epoch 64 (-0.594851) | DKL 0.000451 | BCE 0.712455 | auROC 0.5949 | auPR 0.5846:  16%|█▌        | 79/500 [11:46<1:03:21,  9.03s/it]
[VAL] Best epoch 64 (-0.594851) | DKL 0.000451 | BCE 0.712455 | auROC 0.5949 | auPR 0.5846:  16%|█▌        | 80/500 [11:55<1:02:23,  8.91s/it]
[VAL] Best epoch 64 (-0.594851) | DKL 0.000451 | BCE 0.712455 | auROC 0.5949 | auPR 0.5846:  16%|█▌        | 81/500 [12:04<1:02:12,  8.91s/it]
[VAL] Best epoch 64 (-0.594851) | DKL 0.000451 | BCE 0.712455 | auROC 0.5949 | auPR 0.5846:  16%|█▋        | 82/500 [12:13<1:02:17,  8.94s/it]
[VAL] Best epoch 83 (-0.598404) | DKL 0.000418 | BCE 0.698008 | auROC 0.5984 | auPR 0.5887:  16%|█▋        | 82/500 [12:22<1:02:17,  8.94s/it]
[VAL] Best epoch 83 (-0.598404) | DKL 0.000418 | BCE 0.698008 | auROC 0.5984 | auPR 0.5887:  16%|█▋        | 82/500 [12:22<1:02:17,  8.94s/it]
[VAL] Best epoch 83 (-0.598404) | DKL 0.000418 | BCE 0.698008 | auROC 0.5984 | auPR 0.5887:  17%|█▋        | 83/500 [12:22<1:02:09,  8.94s/it]

[VAL] Best epoch 28 (-0.600209) | DKL 0.000372 | BCE 0.661393 | auROC 0.6002 | auPR 0.5612:   6%|▌         | 28/500 [04:16<1:12:15,  9.19s/it]
[VAL] Best epoch 28 (-0.600209) | DKL 0.000372 | BCE 0.661393 | auROC 0.6002 | auPR 0.5612:   6%|▌         | 29/500 [04:25<1:11:45,  9.14s/it]
[VAL] Best epoch 30 (-0.603260) | DKL 0.000409 | BCE 0.657405 | auROC 0.6033 | auPR 0.5650:   6%|▌         | 29/500 [04:35<1:11:45,  9.14s/it]
[VAL] Best epoch 30 (-0.603260) | DKL 0.000409 | BCE 0.657405 | auROC 0.6033 | auPR 0.5650:   6%|▌         | 29/500 [04:35<1:11:45,  9.14s/it]
[VAL] Best epoch 30 (-0.603260) | DKL 0.000409 | BCE 0.657405 | auROC 0.6033 | auPR 0.5650:   6%|▌         | 30/500 [04:35<1:11:23,  9.11s/it]
[VAL] Best epoch 30 (-0.603260) | DKL 0.000409 | BCE 0.657405 | auROC 0.6033 | auPR 0.5650:   6%|▌         | 31/500 [04:44<1:11:27,  9.14s/it]
[VAL] Best epoch 30 (-0.603260) | DKL 0.000409 | BCE 0.657405 | auROC 0.6033 | auPR 0.5650:   6%|▋         | 32/500 [04:53<1:11:41,  9.19s/it]

[VAL] Best epoch 96 (-0.626035) | DKL 0.000499 | BCE 0.663703 | auROC 0.6260 | auPR 0.6027:  26%|██▌       | 130/500 [19:50<57:25,  9.31s/it]
[VAL] Best epoch 96 (-0.626035) | DKL 0.000499 | BCE 0.663703 | auROC 0.6260 | auPR 0.6027:  26%|██▌       | 131/500 [19:59<56:52,  9.25s/it]
[VAL] Best epoch 96 (-0.626035) | DKL 0.000499 | BCE 0.663703 | auROC 0.6260 | auPR 0.6027:  26%|██▋       | 132/500 [20:08<56:52,  9.27s/it]
[VAL] Best epoch 96 (-0.626035) | DKL 0.000499 | BCE 0.663703 | auROC 0.6260 | auPR 0.6027:  27%|██▋       | 133/500 [20:17<56:26,  9.23s/it]
[VAL] Best epoch 96 (-0.626035) | DKL 0.000499 | BCE 0.663703 | auROC 0.6260 | auPR 0.6027:  27%|██▋       | 134/500 [20:26<55:42,  9.13s/it]
[VAL] Best epoch 96 (-0.626035) | DKL 0.000499 | BCE 0.663703 | auROC 0.6260 | auPR 0.6027:  27%|██▋       | 135/500 [20:35<55:20,  9.10s/it]
[VAL] Best epoch 96 (-0.626035) | DKL 0.000499 | BCE 0.663703 | auROC 0.6260 | auPR 0.6027:  27%|██▋       | 136/500 [20:44<55:22,  9.13s/it]
[VAL] 

[VAL] Best epoch 30 (-0.607133) | DKL 0.000552 | BCE 0.667558 | auROC 0.6071 | auPR 0.5668:  14%|█▍        | 70/500 [10:34<1:04:08,  8.95s/it]
[VAL] Best epoch 30 (-0.607133) | DKL 0.000552 | BCE 0.667558 | auROC 0.6071 | auPR 0.5668:  14%|█▍        | 71/500 [10:43<1:04:31,  9.03s/it]
[VAL] Best epoch 30 (-0.607133) | DKL 0.000552 | BCE 0.667558 | auROC 0.6071 | auPR 0.5668:  14%|█▍        | 72/500 [10:52<1:04:14,  9.01s/it]
[VAL] Best epoch 30 (-0.607133) | DKL 0.000552 | BCE 0.667558 | auROC 0.6071 | auPR 0.5668:  15%|█▍        | 73/500 [11:01<1:04:30,  9.06s/it]
[VAL] Best epoch 30 (-0.607133) | DKL 0.000552 | BCE 0.667558 | auROC 0.6071 | auPR 0.5668:  15%|█▍        | 74/500 [11:10<1:04:28,  9.08s/it]
[VAL] Best epoch 30 (-0.607133) | DKL 0.000552 | BCE 0.667558 | auROC 0.6071 | auPR 0.5668:  15%|█▌        | 75/500 [11:19<1:04:22,  9.09s/it]
[VAL] Best epoch 30 (-0.607133) | DKL 0.000552 | BCE 0.667558 | auROC 0.6071 | auPR 0.5668:  15%|█▌        | 76/500 [11:28<1:04:30,  9.13s/it]

[VAL] Best epoch 57 (-0.626570) | DKL 0.000443 | BCE 0.664811 | auROC 0.6266 | auPR 0.6018:  12%|█▏        | 58/500 [08:10<1:07:03,  9.10s/it]
[VAL] Best epoch 57 (-0.626570) | DKL 0.000443 | BCE 0.664811 | auROC 0.6266 | auPR 0.6018:  12%|█▏        | 59/500 [08:19<1:06:59,  9.11s/it]
[VAL] Best epoch 57 (-0.626570) | DKL 0.000443 | BCE 0.664811 | auROC 0.6266 | auPR 0.6018:  12%|█▏        | 60/500 [08:28<1:06:14,  9.03s/it]
[VAL] Best epoch 61 (-0.627583) | DKL 0.000448 | BCE 0.673579 | auROC 0.6276 | auPR 0.6044:  12%|█▏        | 60/500 [08:37<1:06:14,  9.03s/it]
[VAL] Best epoch 61 (-0.627583) | DKL 0.000448 | BCE 0.673579 | auROC 0.6276 | auPR 0.6044:  12%|█▏        | 60/500 [08:37<1:06:14,  9.03s/it]
[VAL] Best epoch 61 (-0.627583) | DKL 0.000448 | BCE 0.673579 | auROC 0.6276 | auPR 0.6044:  12%|█▏        | 61/500 [08:37<1:06:05,  9.03s/it]
[VAL] Best epoch 61 (-0.627583) | DKL 0.000448 | BCE 0.673579 | auROC 0.6276 | auPR 0.6044:  12%|█▏        | 62/500 [08:46<1:06:35,  9.12s/it]

[VAL] Best epoch 1 (-0.462251) | DKL 0.000162 | BCE 0.696687 | auROC 0.4623 | auPR 0.4746:   0%|          | 2/500 [00:18<1:17:48,  9.37s/it]
[VAL] Best epoch 1 (-0.462251) | DKL 0.000162 | BCE 0.696687 | auROC 0.4623 | auPR 0.4746:   1%|          | 3/500 [00:28<1:17:47,  9.39s/it]
[VAL] Best epoch 4 (-0.521812) | DKL 0.000311 | BCE 0.691344 | auROC 0.5218 | auPR 0.5125:   1%|          | 3/500 [00:37<1:17:47,  9.39s/it]
[VAL] Best epoch 4 (-0.521812) | DKL 0.000311 | BCE 0.691344 | auROC 0.5218 | auPR 0.5125:   1%|          | 3/500 [00:37<1:17:47,  9.39s/it]
[VAL] Best epoch 4 (-0.521812) | DKL 0.000311 | BCE 0.691344 | auROC 0.5218 | auPR 0.5125:   1%|          | 4/500 [00:37<1:17:07,  9.33s/it]
[VAL] Best epoch 4 (-0.521812) | DKL 0.000311 | BCE 0.691344 | auROC 0.5218 | auPR 0.5125:   1%|          | 5/500 [00:46<1:16:22,  9.26s/it]
[VAL] Best epoch 4 (-0.521812) | DKL 0.000311 | BCE 0.691344 | auROC 0.5218 | auPR 0.5125:   1%|          | 6/500 [00:55<1:16:12,  9.26s/it]
[VAL] Best ep

# AVIB - Hard split (Test: only randomized negatives) - Train: negative assays + randomized negatives

In [ ]:
results_avib = []

for i in tqdm(range(5)):
    set_random_seed(i)

    df_train = pd.read_csv(DATA_BASE+"train/only-sampled-negs.full/"+f"train-{i}.csv")
    df_test = pd.read_csv(DATA_BASE+"test/only-sampled-negs/"+f"test-{i}.csv")
    
    scaler = get_st_scaler(df_train, gt='label', cdr3b_col='cdr3.beta', pep_col="antigen.epitope")
    
    # we create a validation set in which the negatives only derive from randomization, like the test set
    df_neg = df_train[df_train['label']==0]
    negative_assays_df = df_neg[df_neg["negative.source"]!='randomized']
    df_train = df_train.drop(negative_assays_df.index)

    inner_train_df, val_df, _ = hard_split_df(
        df_train, target_col="antigen.epitope", min_ratio=0.15, low=500, high=10000
    )
    # at training time the negatives also derive from randomization
    df_train = pd.concat([df_train, negative_assays_df])

    # train
    train_loader = get_balanced_torch_loader(inner_train_df, batch_size, gt='label', pep_col='antigen.epitope', cdr3b_col='cdr3.beta', device=device)
    val_loader = get_balanced_torch_loader(val_df, batch_size, gt='label', pep_col='antigen.epitope', cdr3b_col='cdr3.beta', device=device)

    model = MVIB(z_dim=z_dim, device=device, joint_posterior=joint_posterior).to(device)

    trainer = TrainerMVIB(
        model,
        epochs=epochs,
        lr=lr,
        beta=beta,
        checkpoint_dir=".",
        mode="p+b",
        lr_scheduler_param=lr_scheduler_param
    )
    checkpoint = trainer.train(train_loader, val_loader, early_stopper_patience, monitor)    
    
    # test
    ds_test = TCRDataset(
        df_test,
        torch.device("cpu"),
        cdr3b_col='cdr3.beta',
        pep_col="antigen.epitope",
        gt_col="label",
        cdr3a_col=None,
        scaler=scaler
    )
    model = MVIB.from_checkpoint(checkpoint, cpu_device).to(cpu_device).eval()
    pred = model.classify(pep=ds_test.pep, cdr3b=ds_test.cdr3b, cdr3a=None)
    pred = pred.detach().numpy()
    gt = ds_test.gt.cpu().numpy()
    
    scores_df = get_scores(y_true=gt, y_prob=pred, y_pred=pred.round())
    scores_df['experiment'] = i
    results_avib.append(scores_df)
    df_test['prediction'] = pred
    df_test.to_csv(RESULTS_BASE+f"avib.pep+cdr3b.full.hard-split.{i}.csv", index=False)

results_avib_df = pd.concat(results_avib)
results_avib_df.to_csv(RESULTS_BASE+"avib.pep+cdr3b.full.hard-split.csv", index=False)

  0%|          | 0/5 [00:00<?, ?it/s]/home/nle-fgrazioli/anaconda3/envs/vibtcr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)

  0%|          | 0/500 [00:00<?, ?it/s]
[VAL] Best epoch 1 (-0.499920) | DKL 0.000149 | BCE 0.693525 | auROC 0.4999 | auPR 0.5009:   0%|          | 0/500 [00:08<?, ?it/s]
[VAL] Best epoch 1 (-0.499920) | DKL 0.000149 | BCE 0.693525 | auROC 0.4999 | auPR 0.5009:   0%|          | 0/500 [00:08<?, ?it/s]
[VAL] Best epoch 1 (-0.499920) | DKL 0.000149 | BCE 0.693525 | auROC 0.4999 | auPR 0.5009:   0%|          | 1/500 [00:08<1:08:16,  8.21s/it]
[VAL] Best epoch 1 (-0.499920) | DKL 0.000149 | BCE 0.693525 | auROC 0.4999 | auPR 0.5009:   0%|          | 2/500 [00:16<1:08:05,  8.20s/it]
[VAL] Best epoch 1 (-0.499920) | DKL 0.000149 | BCE 0.693525 | auROC 0.4999 | auPR 0.5009:   1%|   

[VAL] Best epoch 62 (-0.598035) | DKL 0.000469 | BCE 0.702488 | auROC 0.5980 | auPR 0.5810:  16%|█▌        | 81/500 [11:09<58:02,  8.31s/it]
[VAL] Best epoch 62 (-0.598035) | DKL 0.000469 | BCE 0.702488 | auROC 0.5980 | auPR 0.5810:  16%|█▋        | 82/500 [11:18<57:53,  8.31s/it]
[VAL] Best epoch 62 (-0.598035) | DKL 0.000469 | BCE 0.702488 | auROC 0.5980 | auPR 0.5810:  17%|█▋        | 83/500 [11:26<57:40,  8.30s/it]
[VAL] Best epoch 62 (-0.598035) | DKL 0.000469 | BCE 0.702488 | auROC 0.5980 | auPR 0.5810:  17%|█▋        | 84/500 [11:34<57:23,  8.28s/it]
[VAL] Best epoch 62 (-0.598035) | DKL 0.000469 | BCE 0.702488 | auROC 0.5980 | auPR 0.5810:  17%|█▋        | 85/500 [11:42<57:12,  8.27s/it]
[VAL] Best epoch 62 (-0.598035) | DKL 0.000469 | BCE 0.702488 | auROC 0.5980 | auPR 0.5810:  17%|█▋        | 86/500 [11:51<57:00,  8.26s/it]
[VAL] Best epoch 62 (-0.598035) | DKL 0.000469 | BCE 0.702488 | auROC 0.5980 | auPR 0.5810:  17%|█▋        | 87/500 [11:59<56:52,  8.26s/it]
[VAL] Best ep

[VAL] Best epoch 55 (-0.598718) | DKL 0.000380 | BCE 0.668567 | auROC 0.5987 | auPR 0.5681:  11%|█         | 54/500 [07:38<1:01:56,  8.33s/it]
[VAL] Best epoch 55 (-0.598718) | DKL 0.000380 | BCE 0.668567 | auROC 0.5987 | auPR 0.5681:  11%|█         | 55/500 [07:38<1:01:55,  8.35s/it]
[VAL] Best epoch 55 (-0.598718) | DKL 0.000380 | BCE 0.668567 | auROC 0.5987 | auPR 0.5681:  11%|█         | 56/500 [07:47<1:01:58,  8.37s/it]
[VAL] Best epoch 55 (-0.598718) | DKL 0.000380 | BCE 0.668567 | auROC 0.5987 | auPR 0.5681:  11%|█▏        | 57/500 [07:55<1:02:06,  8.41s/it]
[VAL] Best epoch 55 (-0.598718) | DKL 0.000380 | BCE 0.668567 | auROC 0.5987 | auPR 0.5681:  12%|█▏        | 58/500 [08:04<1:01:48,  8.39s/it]
[VAL] Best epoch 55 (-0.598718) | DKL 0.000380 | BCE 0.668567 | auROC 0.5987 | auPR 0.5681:  12%|█▏        | 59/500 [08:12<1:01:40,  8.39s/it]
[VAL] Best epoch 55 (-0.598718) | DKL 0.000380 | BCE 0.668567 | auROC 0.5987 | auPR 0.5681:  12%|█▏        | 60/500 [08:20<1:01:18,  8.36s/it]

[VAL] Best epoch 113 (-0.619294) | DKL 0.000518 | BCE 0.675389 | auROC 0.6193 | auPR 0.5953:  31%|███       | 156/500 [21:43<48:06,  8.39s/it]
[VAL] Best epoch 113 (-0.619294) | DKL 0.000518 | BCE 0.675389 | auROC 0.6193 | auPR 0.5953:  31%|███▏      | 157/500 [21:51<47:48,  8.36s/it]
[VAL] Best epoch 113 (-0.619294) | DKL 0.000518 | BCE 0.675389 | auROC 0.6193 | auPR 0.5953:  32%|███▏      | 158/500 [21:59<47:37,  8.35s/it]
[VAL] Best epoch 113 (-0.619294) | DKL 0.000518 | BCE 0.675389 | auROC 0.6193 | auPR 0.5953:  32%|███▏      | 159/500 [22:08<47:34,  8.37s/it]
[VAL] Best epoch 113 (-0.619294) | DKL 0.000518 | BCE 0.675389 | auROC 0.6193 | auPR 0.5953:  32%|███▏      | 160/500 [22:16<47:21,  8.36s/it]
[VAL] Best epoch 113 (-0.619294) | DKL 0.000518 | BCE 0.675389 | auROC 0.6193 | auPR 0.5953:  32%|███▏      | 161/500 [22:24<47:13,  8.36s/it]
[VAL] Best epoch 113 (-0.619294) | DKL 0.000518 | BCE 0.675389 | auROC 0.6193 | auPR 0.5953:  32%|███▏      | 162/500 [22:41<47:20,  8.40s/it]

[VAL] Best epoch 72 (-0.602751) | DKL 0.000423 | BCE 0.676467 | auROC 0.6028 | auPR 0.5614:  15%|█▍        | 73/500 [10:09<59:40,  8.38s/it]
[VAL] Best epoch 72 (-0.602751) | DKL 0.000423 | BCE 0.676467 | auROC 0.6028 | auPR 0.5614:  15%|█▍        | 74/500 [10:17<59:22,  8.36s/it]
[VAL] Best epoch 72 (-0.602751) | DKL 0.000423 | BCE 0.676467 | auROC 0.6028 | auPR 0.5614:  15%|█▌        | 75/500 [10:25<59:15,  8.36s/it]
[VAL] Best epoch 72 (-0.602751) | DKL 0.000423 | BCE 0.676467 | auROC 0.6028 | auPR 0.5614:  15%|█▌        | 76/500 [10:34<59:06,  8.36s/it]
[VAL] Best epoch 72 (-0.602751) | DKL 0.000423 | BCE 0.676467 | auROC 0.6028 | auPR 0.5614:  15%|█▌        | 77/500 [10:42<58:58,  8.37s/it]
[VAL] Best epoch 78 (-0.603677) | DKL 0.000416 | BCE 0.677523 | auROC 0.6037 | auPR 0.5644:  15%|█▌        | 77/500 [10:50<58:58,  8.37s/it]
[VAL] Best epoch 78 (-0.603677) | DKL 0.000416 | BCE 0.677523 | auROC 0.6037 | auPR 0.5644:  15%|█▌        | 77/500 [10:50<58:58,  8.37s/it]
[VAL] Best ep

[VAL] Best epoch 23 (-0.600204) | DKL 0.000293 | BCE 0.658619 | auROC 0.6002 | auPR 0.5552:   4%|▍         | 22/500 [03:09<1:05:38,  8.24s/it]
[VAL] Best epoch 23 (-0.600204) | DKL 0.000293 | BCE 0.658619 | auROC 0.6002 | auPR 0.5552:   4%|▍         | 22/500 [03:09<1:05:38,  8.24s/it]
[VAL] Best epoch 23 (-0.600204) | DKL 0.000293 | BCE 0.658619 | auROC 0.6002 | auPR 0.5552:   5%|▍         | 23/500 [03:09<1:05:29,  8.24s/it]
[VAL] Best epoch 23 (-0.600204) | DKL 0.000293 | BCE 0.658619 | auROC 0.6002 | auPR 0.5552:   5%|▍         | 24/500 [03:17<1:05:18,  8.23s/it]
[VAL] Best epoch 25 (-0.602442) | DKL 0.000439 | BCE 0.660483 | auROC 0.6024 | auPR 0.5677:   5%|▍         | 24/500 [03:25<1:05:18,  8.23s/it]
[VAL] Best epoch 25 (-0.602442) | DKL 0.000439 | BCE 0.660483 | auROC 0.6024 | auPR 0.5677:   5%|▍         | 24/500 [03:25<1:05:18,  8.23s/it]
[VAL] Best epoch 25 (-0.602442) | DKL 0.000439 | BCE 0.660483 | auROC 0.6024 | auPR 0.5677:   5%|▌         | 25/500 [03:25<1:04:59,  8.21s/it]

[VAL] Best epoch 87 (-0.641377) | DKL 0.000541 | BCE 0.674829 | auROC 0.6414 | auPR 0.6187:  23%|██▎       | 115/500 [15:38<50:38,  7.89s/it]
[VAL] Best epoch 87 (-0.641377) | DKL 0.000541 | BCE 0.674829 | auROC 0.6414 | auPR 0.6187:  23%|██▎       | 116/500 [15:46<50:35,  7.90s/it]
[VAL] Best epoch 87 (-0.641377) | DKL 0.000541 | BCE 0.674829 | auROC 0.6414 | auPR 0.6187:  23%|██▎       | 117/500 [15:54<50:27,  7.90s/it]
[VAL] Best epoch 87 (-0.641377) | DKL 0.000541 | BCE 0.674829 | auROC 0.6414 | auPR 0.6187:  24%|██▎       | 118/500 [16:02<50:26,  7.92s/it]
[VAL] Best epoch 87 (-0.641377) | DKL 0.000541 | BCE 0.674829 | auROC 0.6414 | auPR 0.6187:  24%|██▍       | 119/500 [16:10<50:16,  7.92s/it]
[VAL] Best epoch 87 (-0.641377) | DKL 0.000541 | BCE 0.674829 | auROC 0.6414 | auPR 0.6187:  24%|██▍       | 120/500 [16:18<50:09,  7.92s/it]
[VAL] Best epoch 87 (-0.641377) | DKL 0.000541 | BCE 0.674829 | auROC 0.6414 | auPR 0.6187:  24%|██▍       | 121/500 [16:26<49:54,  7.90s/it]
[VAL] 

[VAL] Best epoch 22 (-0.517064) | DKL 0.000447 | BCE 0.710066 | auROC 0.5171 | auPR 0.5250:   6%|▌         | 28/500 [03:44<1:03:00,  8.01s/it]
[VAL] Best epoch 22 (-0.517064) | DKL 0.000447 | BCE 0.710066 | auROC 0.5171 | auPR 0.5250:   6%|▌         | 29/500 [03:52<1:02:50,  8.00s/it]
[VAL] Best epoch 22 (-0.517064) | DKL 0.000447 | BCE 0.710066 | auROC 0.5171 | auPR 0.5250:   6%|▌         | 30/500 [04:00<1:02:49,  8.02s/it]
[VAL] Best epoch 22 (-0.517064) | DKL 0.000447 | BCE 0.710066 | auROC 0.5171 | auPR 0.5250:   6%|▌         | 31/500 [04:08<1:02:37,  8.01s/it]
[VAL] Best epoch 22 (-0.517064) | DKL 0.000447 | BCE 0.710066 | auROC 0.5171 | auPR 0.5250:   6%|▋         | 32/500 [04:16<1:02:27,  8.01s/it]
[VAL] Best epoch 22 (-0.517064) | DKL 0.000447 | BCE 0.710066 | auROC 0.5171 | auPR 0.5250:   7%|▋         | 33/500 [04:24<1:02:20,  8.01s/it]
[VAL] Best epoch 22 (-0.517064) | DKL 0.000447 | BCE 0.710066 | auROC 0.5171 | auPR 0.5250:   7%|▋         | 34/500 [04:32<1:02:11,  8.01s/it]

# NetTCR-2.0 - Random split - Train and test: only randomized negatives

In [6]:
results_avib = []

for i in tqdm(range(5)):
    set_random_seed(i)

    df = pd.read_csv(f"/home/{login}/Git/tc-hard/tc-hard-data/tc-hard/ds.csv")
    df['label']=df['label'].astype('int64')

    df = df[df["negative.source"] != "mira"]
    df = df[df["negative.source"] != "iedb"]
    df = df[df["negative.source"] != "nettcr-2.0"]

    df = df.drop_duplicates(
        subset=["antigen.epitope", "cdr3.beta", "label"], keep="first"
    ).reset_index(drop=True)
    
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=i)
    
    scaler = get_st_scaler(df_train, gt='label', cdr3b_col='cdr3.beta', pep_col="antigen.epitope")
    inner_train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=i)
    
    # train
    train_loader = get_balanced_torch_loader(inner_train_df, batch_size, gt='label', pep_col='antigen.epitope', cdr3b_col='cdr3.beta', device=device)
    val_loader = get_balanced_torch_loader(val_df, batch_size, gt='label', pep_col='antigen.epitope', cdr3b_col='cdr3.beta', device=device)

    model = MVIB(z_dim=z_dim, device=device, joint_posterior=joint_posterior).to(device)

    trainer = TrainerMVIB(
        model,
        epochs=epochs,
        lr=lr,
        beta=beta,
        checkpoint_dir=".",
        mode="p+b",
        lr_scheduler_param=lr_scheduler_param
    )
    checkpoint = trainer.train(train_loader, val_loader, early_stopper_patience, monitor)    
    
    # test
    ds_test = TCRDataset(
        df_test,
        torch.device("cpu"),
        cdr3b_col='cdr3.beta',
        pep_col="antigen.epitope",
        gt_col="label",
        cdr3a_col=None,
        scaler=scaler
    )
    model = MVIB.from_checkpoint(checkpoint, cpu_device).to(cpu_device).eval()
    pred = model.classify(pep=ds_test.pep, cdr3b=ds_test.cdr3b, cdr3a=None)
    pred = pred.detach().numpy()
    gt = ds_test.gt.cpu().numpy()
    
    scores_df = get_scores(y_true=gt, y_prob=pred, y_pred=pred.round())
    scores_df['experiment'] = i
    results_avib.append(scores_df)
    df_test['prediction'] = pred
    df_test.to_csv(RESULTS_BASE+f"avib.pep+cdr3b.only-sampled-negs.random-split.{i}.csv", index=False)

results_avib_df = pd.concat(results_avib)
results_avib_df.to_csv(RESULTS_BASE+"avib.pep+cdr3b.only-sampled-negs.random-split.csv", index=False)

  0%|          | 0/5 [00:00<?, ?it/s]/home/nle-fgrazioli/anaconda3/envs/vibtcr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)

  0%|          | 0/500 [00:00<?, ?it/s]
[VAL] Best epoch 1 (-0.545894) | DKL 0.000159 | BCE 0.686473 | auROC 0.5459 | auPR 0.5597:   0%|          | 0/500 [00:07<?, ?it/s]
[VAL] Best epoch 1 (-0.545894) | DKL 0.000159 | BCE 0.686473 | auROC 0.5459 | auPR 0.5597:   0%|          | 0/500 [00:07<?, ?it/s]
[VAL] Best epoch 1 (-0.545894) | DKL 0.000159 | BCE 0.686473 | auROC 0.5459 | auPR 0.5597:   0%|          | 1/500 [00:07<1:05:01,  7.82s/it]
[VAL] Best epoch 2 (-0.554259) | DKL 0.000134 | BCE 0.685309 | auROC 0.5543 | auPR 0.5665:   0%|          | 1/500 [00:15<1:05:01,  7.82s/it]
[VAL] Best epoch 2 (-0.554259) | DKL 0.000134 | BCE 0.685309 | auROC 0.5543 | auPR 0.5665:   0%|   

[VAL] Best epoch 21 (-0.718325) | DKL 0.000586 | BCE 0.600453 | auROC 0.7183 | auPR 0.7290:   4%|▍         | 21/500 [02:42<1:02:43,  7.86s/it]
[VAL] Best epoch 22 (-0.718985) | DKL 0.000685 | BCE 0.600758 | auROC 0.7190 | auPR 0.7303:   4%|▍         | 21/500 [02:51<1:02:43,  7.86s/it]
[VAL] Best epoch 22 (-0.718985) | DKL 0.000685 | BCE 0.600758 | auROC 0.7190 | auPR 0.7303:   4%|▍         | 21/500 [02:51<1:02:43,  7.86s/it]
[VAL] Best epoch 22 (-0.718985) | DKL 0.000685 | BCE 0.600758 | auROC 0.7190 | auPR 0.7303:   4%|▍         | 22/500 [02:51<1:03:17,  7.94s/it]
[VAL] Best epoch 22 (-0.718985) | DKL 0.000685 | BCE 0.600758 | auROC 0.7190 | auPR 0.7303:   5%|▍         | 23/500 [02:58<1:02:19,  7.84s/it]
[VAL] Best epoch 24 (-0.719841) | DKL 0.000526 | BCE 0.600062 | auROC 0.7198 | auPR 0.7301:   5%|▍         | 23/500 [03:06<1:02:19,  7.84s/it]
[VAL] Best epoch 24 (-0.719841) | DKL 0.000526 | BCE 0.600062 | auROC 0.7198 | auPR 0.7301:   5%|▍         | 23/500 [03:06<1:02:19,  7.84s/it]

[VAL] Best epoch 47 (-0.750127) | DKL 0.000529 | BCE 0.576361 | auROC 0.7501 | auPR 0.7627:   9%|▉         | 47/500 [06:00<57:11,  7.58s/it]
[VAL] Best epoch 48 (-0.752309) | DKL 0.000549 | BCE 0.572666 | auROC 0.7523 | auPR 0.7663:   9%|▉         | 47/500 [06:08<57:11,  7.58s/it]
[VAL] Best epoch 48 (-0.752309) | DKL 0.000549 | BCE 0.572666 | auROC 0.7523 | auPR 0.7663:   9%|▉         | 47/500 [06:08<57:11,  7.58s/it]
[VAL] Best epoch 48 (-0.752309) | DKL 0.000549 | BCE 0.572666 | auROC 0.7523 | auPR 0.7663:  10%|▉         | 48/500 [06:08<57:03,  7.57s/it]
[VAL] Best epoch 49 (-0.754297) | DKL 0.000543 | BCE 0.572171 | auROC 0.7543 | auPR 0.7655:  10%|▉         | 48/500 [06:15<57:03,  7.57s/it]
[VAL] Best epoch 49 (-0.754297) | DKL 0.000543 | BCE 0.572171 | auROC 0.7543 | auPR 0.7655:  10%|▉         | 48/500 [06:15<57:03,  7.57s/it]
[VAL] Best epoch 49 (-0.754297) | DKL 0.000543 | BCE 0.572171 | auROC 0.7543 | auPR 0.7655:  10%|▉         | 49/500 [06:15<56:59,  7.58s/it]
[VAL] Best ep

[VAL] Best epoch 80 (-0.767490) | DKL 0.000537 | BCE 0.561972 | auROC 0.7675 | auPR 0.7811:  16%|█▌        | 79/500 [10:10<53:13,  7.58s/it]
[VAL] Best epoch 80 (-0.767490) | DKL 0.000537 | BCE 0.561972 | auROC 0.7675 | auPR 0.7811:  16%|█▌        | 80/500 [10:10<53:06,  7.59s/it]
[VAL] Best epoch 81 (-0.771168) | DKL 0.000549 | BCE 0.557250 | auROC 0.7712 | auPR 0.7831:  16%|█▌        | 80/500 [10:18<53:06,  7.59s/it]
[VAL] Best epoch 81 (-0.771168) | DKL 0.000549 | BCE 0.557250 | auROC 0.7712 | auPR 0.7831:  16%|█▌        | 80/500 [10:18<53:06,  7.59s/it]
[VAL] Best epoch 81 (-0.771168) | DKL 0.000549 | BCE 0.557250 | auROC 0.7712 | auPR 0.7831:  16%|█▌        | 81/500 [10:18<52:51,  7.57s/it]
[VAL] Best epoch 81 (-0.771168) | DKL 0.000549 | BCE 0.557250 | auROC 0.7712 | auPR 0.7831:  16%|█▋        | 82/500 [10:25<52:49,  7.58s/it]
[VAL] Best epoch 81 (-0.771168) | DKL 0.000549 | BCE 0.557250 | auROC 0.7712 | auPR 0.7831:  17%|█▋        | 83/500 [10:33<52:40,  7.58s/it]
[VAL] Best ep

[VAL] Best epoch 125 (-0.776876) | DKL 0.000606 | BCE 0.554684 | auROC 0.7769 | auPR 0.7872:  25%|██▌       | 125/500 [15:51<47:23,  7.58s/it]
[VAL] Best epoch 126 (-0.777192) | DKL 0.000544 | BCE 0.554757 | auROC 0.7772 | auPR 0.7893:  25%|██▌       | 125/500 [15:59<47:23,  7.58s/it]
[VAL] Best epoch 126 (-0.777192) | DKL 0.000544 | BCE 0.554757 | auROC 0.7772 | auPR 0.7893:  25%|██▌       | 125/500 [15:59<47:23,  7.58s/it]
[VAL] Best epoch 126 (-0.777192) | DKL 0.000544 | BCE 0.554757 | auROC 0.7772 | auPR 0.7893:  25%|██▌       | 126/500 [15:59<47:16,  7.58s/it]
[VAL] Best epoch 126 (-0.777192) | DKL 0.000544 | BCE 0.554757 | auROC 0.7772 | auPR 0.7893:  25%|██▌       | 127/500 [16:06<47:25,  7.63s/it]
[VAL] Best epoch 126 (-0.777192) | DKL 0.000544 | BCE 0.554757 | auROC 0.7772 | auPR 0.7893:  26%|██▌       | 128/500 [16:14<47:15,  7.62s/it]
[VAL] Best epoch 126 (-0.777192) | DKL 0.000544 | BCE 0.554757 | auROC 0.7772 | auPR 0.7893:  26%|██▌       | 129/500 [16:22<47:04,  7.61s/it]

...

[VAL] Best epoch 64 (-0.758955) | DKL 0.000582 | BCE 0.566525 | auROC 0.7590 | auPR 0.7710:  13%|█▎        | 66/500 [09:02<55:57,  7.74s/it]
[VAL] Best epoch 64 (-0.758955) | DKL 0.000582 | BCE 0.566525 | auROC 0.7590 | auPR 0.7710:  13%|█▎        | 67/500 [09:09<55:50,  7.74s/it]
[VAL] Best epoch 64 (-0.758955) | DKL 0.000582 | BCE 0.566525 | auROC 0.7590 | auPR 0.7710:  14%|█▎        | 68/500 [09:17<55:37,  7.73s/it]
[VAL] Best epoch 64 (-0.758955) | DKL 0.000582 | BCE 0.566525 | auROC 0.7590 | auPR 0.7710:  14%|█▍        | 69/500 [09:25<55:31,  7.73s/it]
[VAL] Best epoch 64 (-0.758955) | DKL 0.000582 | BCE 0.566525 | auROC 0.7590 | auPR 0.7710:  14%|█▍        | 70/500 [09:32<55:24,  7.73s/it]
[VAL] Best epoch 64 (-0.758955) | DKL 0.000582 | BCE 0.566525 | auROC 0.7590 | auPR 0.7710:  14%|█▍        | 71/500 [09:40<55:13,  7.72s/it]
[VAL] Best epoch 64 (-0.758955) | DKL 0.000582 | BCE 0.566525 | auROC 0.7590 | auPR 0.7710:  14%|█▍        | 72/500 [09:48<55:09,  7.73s/it]
[VAL] Best ep

[VAL] Best epoch 102 (-0.771264) | DKL 0.000595 | BCE 0.557747 | auROC 0.7713 | auPR 0.7837:  21%|██        | 104/500 [13:55<51:03,  7.74s/it]
[VAL] Best epoch 102 (-0.771264) | DKL 0.000595 | BCE 0.557747 | auROC 0.7713 | auPR 0.7837:  21%|██        | 105/500 [14:03<51:01,  7.75s/it]
[VAL] Best epoch 102 (-0.771264) | DKL 0.000595 | BCE 0.557747 | auROC 0.7713 | auPR 0.7837:  21%|██        | 106/500 [14:11<50:54,  7.75s/it]
[VAL] Best epoch 102 (-0.771264) | DKL 0.000595 | BCE 0.557747 | auROC 0.7713 | auPR 0.7837:  21%|██▏       | 107/500 [14:19<50:44,  7.75s/it]
[VAL] Best epoch 108 (-0.772509) | DKL 0.000598 | BCE 0.557731 | auROC 0.7725 | auPR 0.7851:  21%|██▏       | 107/500 [14:26<50:44,  7.75s/it]
[VAL] Best epoch 108 (-0.772509) | DKL 0.000598 | BCE 0.557731 | auROC 0.7725 | auPR 0.7851:  21%|██▏       | 107/500 [14:26<50:44,  7.75s/it]
[VAL] Best epoch 108 (-0.772509) | DKL 0.000598 | BCE 0.557731 | auROC 0.7725 | auPR 0.7851:  22%|██▏       | 108/500 [14:26<50:37,  7.75s/it]

[VAL] Best epoch 144 (-0.778191) | DKL 0.000573 | BCE 0.553938 | auROC 0.7782 | auPR 0.7874:  30%|███       | 152/500 [20:07<44:51,  7.74s/it]
[VAL] Best epoch 153 (-0.779297) | DKL 0.000640 | BCE 0.553753 | auROC 0.7793 | auPR 0.7895:  30%|███       | 152/500 [20:15<44:51,  7.74s/it]
[VAL] Best epoch 153 (-0.779297) | DKL 0.000640 | BCE 0.553753 | auROC 0.7793 | auPR 0.7895:  30%|███       | 152/500 [20:15<44:51,  7.74s/it]
[VAL] Best epoch 153 (-0.779297) | DKL 0.000640 | BCE 0.553753 | auROC 0.7793 | auPR 0.7895:  31%|███       | 153/500 [20:15<44:46,  7.74s/it]
[VAL] Best epoch 153 (-0.779297) | DKL 0.000640 | BCE 0.553753 | auROC 0.7793 | auPR 0.7895:  31%|███       | 154/500 [20:22<44:42,  7.75s/it]
[VAL] Best epoch 153 (-0.779297) | DKL 0.000640 | BCE 0.553753 | auROC 0.7793 | auPR 0.7895:  31%|███       | 155/500 [20:30<44:33,  7.75s/it]
[VAL] Best epoch 153 (-0.779297) | DKL 0.000640 | BCE 0.553753 | auROC 0.7793 | auPR 0.7895:  31%|███       | 156/500 [20:38<44:25,  7.75s/it]

[VAL] Best epoch 203 (-0.781405) | DKL 0.000652 | BCE 0.555500 | auROC 0.7814 | auPR 0.7958:  40%|████      | 202/500 [26:41<38:25,  7.74s/it]
[VAL] Best epoch 203 (-0.781405) | DKL 0.000652 | BCE 0.555500 | auROC 0.7814 | auPR 0.7958:  41%|████      | 203/500 [26:41<38:16,  7.73s/it]
[VAL] Best epoch 203 (-0.781405) | DKL 0.000652 | BCE 0.555500 | auROC 0.7814 | auPR 0.7958:  41%|████      | 204/500 [26:49<38:10,  7.74s/it]
[VAL] Best epoch 205 (-0.782930) | DKL 0.000605 | BCE 0.554569 | auROC 0.7829 | auPR 0.7935:  41%|████      | 204/500 [26:57<38:10,  7.74s/it]
[VAL] Best epoch 205 (-0.782930) | DKL 0.000605 | BCE 0.554569 | auROC 0.7829 | auPR 0.7935:  41%|████      | 204/500 [26:57<38:10,  7.74s/it]
[VAL] Best epoch 205 (-0.782930) | DKL 0.000605 | BCE 0.554569 | auROC 0.7829 | auPR 0.7935:  41%|████      | 205/500 [26:57<38:03,  7.74s/it]
[VAL] Best epoch 205 (-0.782930) | DKL 0.000605 | BCE 0.554569 | auROC 0.7829 | auPR 0.7935:  41%|████      | 206/500 [27:05<37:51,  7.73s/it]

100%|██████████| 5/5 [2:28:28<00:00, 1781.68s/it]
